Importando as matérias do curso de Mecatrônica, processando-os e adicionando uma coluna de "Documentos" no Dataframe das Matérias:

Esse dataframe será salvo no arquivo: "documents_df.txt" no formato binário do Pickle.

In [1]:
import numpy as np
import pandas as pd
import pickle
import gensim
import spacy

from nltk.corpus import stopwords

nlp = spacy.load('pt_core_news_lg')

# * adding custom texts that dont represent real words
noises_list = ["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x", "xi"]

stopWords_list = stopwords.words("portuguese")

# * adding custom words to StopWords list
stopWords_list += [
    'referente',
    'seguinte',
    'etc',
    'ª',
    'tal',
    'um', 
    'dois',
    'tres',
    'vs',
    'aula',
    'tal',
]

# * preprocessing stopwords to correct format
stopWords_list = gensim.utils.simple_preprocess(" ".join(stopWords_list), deacc=True, min_len=1, max_len=40)

# * manual intervention, changing final lemmas
intervention_dict = {
    "campar": "campo",
    "seriar":"serie",
    "eletromagnetico":"eletromagnetismo",
}

def preprocess(text):
    # * importing stopwords from nltk and spacy pipeline
    global nlp
    global stopWords_list
    global noises_list
    global intervention_dict

    # * preprocessing text with gensim.simple_preprocess, eliminating noises: lowercase, tokenized, no symbols, no numbers, no accents marks(normatize)
    text_list = gensim.utils.simple_preprocess(text, deacc=True, min_len=1, max_len=40)

    # * recombining tokens to a string type object and removing remaining noises
    text_str = " ".join([word for word in text_list if word not in noises_list])

    # * preprocessing with spacy, retokenizing -> tagging parts of speech (PoS) -> parsing (assigning dependencies between words) -> lemmatizing
    text_doc = nlp(text_str)

    # * re-tokenization, removing stopwords and lemmatizing
    lemmatized_text_list = [token.lemma_ for token in text_doc if token.text not in stopWords_list]

    # * manual intervention conversion of lemmas
    output = []
    for token in lemmatized_text_list:
        if token in intervention_dict:
            output.append(intervention_dict[token])
        else:
            output.append(token)
            
    return output

subjects_df = pd.read_json("MechatronicsEngeneeringSubjects.json")

documents_list = []

for i, row in subjects_df.iterrows():

    # * reading values of each subject (row)
    subject_id = row["codigo"]
    name = row["nome"]
    syllabus = row["ementa"]
    content = row["conteudo"]

    # * combining them to create the subject document
    text = name + ' ' + syllabus + ' ' + content
    
    # * preprocessing
    preProcessedText = preprocess(text)
    documents_list.append(preProcessedText)

documents_series = pd.Series(documents_list, name="documento")

documents_df = pd.concat([subjects_df, documents_series], axis=1)

with open('documents_df.txt', 'wb') as f:
    pickle.dump(documents_df, f)

# * crea topics
crea_df = pd.read_excel("Matriz_do_Conhecimento.xls", skiprows=4).iloc[:, :9].fillna(method="ffill")
crea_df = crea_df.replace({"TÓPICOS": np.NaN, "Nº DE ORDEM DOS TÓPICOS": np.NaN})

with open('crea_df_pickle.txt', 'wb') as f:
    pickle.dump(crea_df, f)

documents_df

D:\ANACONDA\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


,codigo,nome,ementa,conteudo,documento
0,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES,Princípios fundamentais de construção de progr...,,"[algoritmo, programacao, computador, principio..."
1,IFD0171,FISICA 1,Módulos 1: Unidades e grandezas físicas 2: Vet...,"I-MEDICAO: GRANDEZAS, PADROES E UNIDADES FISIC...","[fisica, modulos, unidade, grandeza, fisicas, ..."
2,IFD0173,FISICA 1 EXPERIMENTAL,MEDIDAS E ERROS. ANALISE GRAFICA. ATRITO. COLI...,I-CLASSIFICACAO DOS ERROS. CALCULO DE ERRO EXP...,"[fisica, experimental, medir, erro, analisar, ..."
3,MAT0025,CÁLCULO 1,"Funções de uma variável real, limite e continu...",1. Funções: conceito de função exemplo de funç...,"[calcular, funcoes, variavel, real, limitar, c..."
4,ENM0133,INTRODUCAO A ENGENHARIA MECATRONICA,"A Universidade de Brasília. Engenharia, Contro...",,"[introducao, engenhar, mecatronica, universida..."
5,IQD0125,QUIMICA GERAL TEORICA,Abordagem conceitual dos princípios fundamenta...,1.Estrutura Atômica e a Lei Periódica: O Model...,"[quimica, geral, teorica, abordagem, conceitua..."
6,IQD0126,QUIMICA GERAL EXPERIMENTAL,CARACTERIZACAO DA NATUREZA E DO PAPEL DAS INVE...,1) Noções Básicas sobre Segurança no Trabalho ...,"[quimica, geral, experimental, caracterizacao,..."
7,ENM0131,DESENHO MECANICO ASSISTIDO POR COMPUTADOR 1,Objetivos da Disciplina: A disciplina objetiva...,- Normas de desenho Técnico- Introdução ao CAD...,"[desenhar, mecanico, assistir, computador, obj..."
8,EST0023,PROBABILIDADE E ESTATÍSTICA,"Análise Descritiva, Cálculo de Probabilidades,...",UNIDADE I - Análise Descritiva de Dados - Popu...,"[probabilidade, estatistica, analisar, descrit..."
9,IFD0175,FISICA 2,Módulos 1: Equilíbrio e elasticidade 2: Gravit...,1-CONTEUDO TEORICO 1-DINAMICA DA ROTACAO 1.1- ...,"[fisica, modulos, equilibrio, elasticidade, gr..."
